# 0.0 Imports

In [19]:
import pandas as pd
import numpy as np
import inflection
from geopy.geocoders import Nominatim

In [27]:
# Inicializa o geolocalizador
geolocator = Nominatim(user_agent="myGeocoder")

# Função para pegar latitude e longitude de uma cidade
def get_lat_lon(city_name):
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

In [2]:
data = pd.read_excel('../data/planilha_vendas.xlsx')

# 1.0 Descrição dos Dados

In [3]:
df1 = data.copy()

## 1.1. Tamanho dos Dados

In [5]:
print(f'Numero de linhas é de: {df1.shape[0]}')
print(f'Numero de linhas é de: {df1.shape[1]}')

Numero de linhas é de: 191
Numero de linhas é de: 16


## 1.2. Coluna dos Dados

In [9]:
old_cols = ['Emissor', 'No Pedido', 'Revenda', 'Cidade', 'Produto', 'Quantidade',
       'Preço', 'Total', 'Data de emissão', 'Data de Faturamento',
       'Tipo de Vencimento', 'Data de Vencimento', '% Comissão',
       'Comissão Total', 'Comissão 1', 'Comissão 2']

snakecase = lambda x: inflection.underscore( x )

new_cols = list( map( snakecase, old_cols ) )

df1.columns = new_cols
df1.columns

Index(['emissor', 'no pedido', 'revenda', 'cidade', 'produto', 'quantidade',
       'preço', 'total', 'data de emissão', 'data de faturamento',
       'tipo de vencimento', 'data de vencimento', '% comissão',
       'comissão total', 'comissão 1', 'comissão 2'],
      dtype='object')

## 1.3. Data Types

In [10]:
df1.dtypes

emissor                        object
no pedido                      object
revenda                        object
cidade                         object
produto                        object
quantidade                      int64
preço                         float64
total                         float64
data de emissão        datetime64[ns]
data de faturamento    datetime64[ns]
tipo de vencimento             object
data de vencimento     datetime64[ns]
% comissão                    float64
comissão total                float64
comissão 1                    float64
comissão 2                    float64
dtype: object

## 1.4. Check NA

In [12]:
df1.isna().sum()

emissor                  0
no pedido               19
revenda                  0
cidade                   0
produto                  0
quantidade               0
preço                    0
total                    0
data de emissão          0
data de faturamento    101
tipo de vencimento      23
data de vencimento       4
% comissão               0
comissão total           0
comissão 1               0
comissão 2               0
dtype: int64

# 2.0. Engenharia de Features

In [31]:
df2 = df1.copy()

## 2.1. Ano e Mes

In [32]:
# year
df2['ano'] = df2['data de emissão'].dt.year

# month
df2['mes'] = df2['data de emissão'].dt.month

## 2.2. Produto Geral

In [33]:
df2['produto_geral'] = df2['produto'].apply(lambda x: x.split()[0])

## 2.3. Latitude e Longitude

In [35]:
# Criação das colunas 'latitude' e 'longitude'
df2[['latitude', 'longitude']] = df2['cidade'].apply(lambda city: pd.Series(get_lat_lon(city)))

# 3.0 Selecionar Colunas

In [39]:
feature_vendas = df2.loc[:, ['revenda', 'cidade', 'produto', 'quantidade', 'preço', 'total', 'ano', 'mes', 'produto_geral', 'latitude', 'longitude']]

In [40]:
feature_vendas.head()

,revenda,cidade,produto,quantidade,preço,total,ano,mes,produto_geral,latitude,longitude
0,TDM Insumos,Ipuiúna,Moximate WP 1x20kg,2560,38.0,97280.0,2022,6,Moximate,-22.099320,-46.189656
1,TDM Insumos,Ipuiúna,Xopotó 800 WP 1x10kg,5120,19.2,98304.0,2022,6,Xopotó,-22.099320,-46.189656
2,Md Agro,Pouso Alegre,Moximate WP 1x20kg,8960,38.0,340480.0,2022,6,Moximate,-22.234386,-45.932724
3,Md Agro,Pouso Alegre,Moximate WP 1x10kg,5120,39.0,199680.0,2022,6,Moximate,-22.234386,-45.932724
4,Md Agro,Pouso Alegre,Moximate WP 10x1kg,3240,40.0,129600.0,2022,6,Moximate,-22.234386,-45.932724
